# Task 4: Demonstrating Quantum Advantage

In [ ]:
"""
This task is intentionally open-ended, so no boilerplate code is provided.

You may use this notebook to develop your solution, or create a separate file if you prefer.
We recommend starting by copying over your previous implementations of the QRNG, TRNG, and PRNG.
Then, explore ways to modularize and combine these components to design a use case that
demonstrates the unique advantages of quantum randomness.

Your write-up can be included directly in this notebook or submitted separately.
You're welcome to prepare it as a Google Doc or LaTeX document and upload a PDF to the GitHub repository—
just be sure to clearly indicate where it can be found if it's not included here.
"""

In [6]:
%pip install requests
%pip install numpy
%pip install matplotlib
%pip install pyaudio
%pip install psutil
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      /private/var/folders/f5/299vphyd6fz0xl9_z3gz0w7h0000gn/T/pip-build-env-vd_vx0di/overlay/lib/python3.12/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI

In [7]:
# Package Imports - feel free to add what you think might be useful! 
import requests
import time
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import hashlib
import struct
import psutil
import platform
import pandas as pd

In [10]:
def get_qrng_floats(n, min_val=-32768, max_val=32767):
    if n > 128:
        n = 128
    url = "https://qrng.idqloud.com/api/1.0/short"
    headers = {"X-API-KEY": "aTo4BKRvnc49uRWDk034zaua87vGRXKk9TMLdfkI"}
    params = {"min": min_val, "max": max_val, "quantity": n}
    response = requests.get(url, headers=headers, params=params)
    raw = np.array(response.json()['data'], dtype=np.float64)
    scaled = raw - min_val
    scaled = scaled / (abs(max_val) + abs(min_val)) 

    #Random numbers scale to [0,1)
    return scaled

In [ ]:
def get_laplace_transform(P, delta, epsilon):
    #P is the scaled random numbers from get_qrng_floats()
    #Delta is the probability that for any one query, data is leaked (generally 1/N)
    #Epsilon is the threshold for privacy (smaller epsilon = more privacy = less utility)
    b = delta/epsilon
    laplace = []
    for p in P:
        if p == 0:
            laplace.append(float('-inf'))
        elif p < 0.5:
            laplace.append(b * math.log(2 * p))
        else:
            laplace.append(-b * math.log(2 * (1 - p)))
    return laplace

In [6]:
get_qrng_floats(10)

array([0.38355077, 0.42789349, 0.66877241, 0.15347524, 0.04206912,
       0.78310826, 0.12239261, 0.86601053, 0.84707408, 0.74633402])

In [14]:
size = 20

positive = 324 + np.random.normal(0, 4, size).astype(int).cumsum()
total = 10000 - np.random.poisson(2, size).cumsum()

In [19]:
hiv_df = pd.DataFrame(np.array([positive, total]).T, columns=["Positive", "Total"])
hiv_df.index = range(2000, 2020)

In [20]:
display(hiv_df)

,Positive,Total
2000,318,9998
2001,319,9996
2002,322,9995
2003,325,9992
2004,322,9990
2005,321,9989
2006,320,9989
2007,318,9987
2008,317,9986
2009,312,9985
